In [1]:
# Import the necessary packages
import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import time

In [2]:
# Defining the local environment variable with the key received in the previous step
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAOrPYAEAAAAAzotm%2BluowpF8blMB3kwWGXk%2B%2BJU%3DrKoUdWu5nquE4NaYEQ27TJVo103unUVbW5JQmY8L4GIZckjHwp'
headers = {"Authorization": "Bearer {}".format(os.getenv('TOKEN'))}

# Defining the Academic Research URL
search_url = "https://api.twitter.com/2/tweets/search/all" 

# creating url with query params
def create_url(keyword, start_date, end_date, max_results = 10):
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,  
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,public_metrics',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return query_params




In [3]:
# Connect to the end-point and send a request to the API
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   
    response = requests.request("GET", url, headers = headers, params = params)

    # Raise an exception if the request didn't succeeded
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [4]:
# Inputs for the request
keyword = "vaccine lang:en place_country:US"
start_time = "2021-03-01T00:00:00.000Z"
end_time = "2021-03-31T00:00:00.000Z"
max_results = 10

# Make a request
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(search_url, headers, url)

print(json_response)

{'data': [{'conversation_id': '1377019164448153600', 'id': '1377047219036155904', 'text': '@DeondreJackson_ also vaccine prevents 100% of hospitalization due the covid \n\nAnd frankly any hosptilization in this country is gonna cost you ATLEAST 10k and a massive headache with billing. She want that bill? I’ll take one day of side effects to never step foot in a hospital again', 'geo': {'place_id': '590c1000bfb45cf6'}, 'in_reply_to_user_id': '3161427649', 'author_id': '3161427649', 'created_at': '2021-03-30T23:56:55.000Z', 'public_metrics': {'retweet_count': 0, 'reply_count': 0, 'like_count': 3, 'quote_count': 0}}, {'conversation_id': '1377047210735644677', 'id': '1377047210735644677', 'text': 'Next week the state of Delaware is expanding vaccine eligibility to all Delawareans 16 and older. #decovid #CovidVaccine #COVID19 #pandemia #Salud https://t.co/6ZuAgLW3u4', 'geo': {'place_id': '3f5897b87d2bf56c'}, 'author_id': '46679143', 'created_at': '2021-03-30T23:56:53.000Z', 'public_metrics'

In [5]:
# Create a new CSV file
csvFile = open("twitter_data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

# Create features titles
csvWriter.writerow(['text', 'author_id', 'created_at', 'geo', 'tweet_id', 'retweet_count'])
csvFile.close()

In [6]:
def append_to_csv(json_response):

    # Open the CSV file
    csvFile = open("twitter_data.csv", "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)
    
    # Get list of places from json_response
    places = {p["id"]: p for p in json_response["includes"]['places']}

    # Loop through each tweet in the json_response 
    for tweet in json_response['data']:
        text = tweet['text']
        author_id = tweet['author_id']
        created_at = dateutil.parser.parse(tweet['created_at'])
        tweet_id = tweet['id']
        retweet_count = tweet['public_metrics']['retweet_count']

        try:
            geo =places[tweet["geo"]['place_id']]["full_name"]
        except:
            geo = "None"
        
        # Append the tweet to a new row
        csvWriter.writerow([text, author_id, created_at, geo, tweet_id, retweet_count])

    csvFile.close()

In [9]:
keyword = "vaccines lang:en place_country:US OR vaccine lang:en place_country:US"   
start_list =    ['2021-01-01T00:00:00.000Z']
end_list =      ['2021-07-31T00:00:00.000Z']
max_results = 500


# Initialzing the total collected tweets
count = 0
# Defining the total tweets collected limit
max_count = 100000
# Initialzing the first next_token
next_token = None

while True:
    # Check if max_count reached
    if count >= max_count:
        break

    # Make a request
    url = create_url(keyword, start_list[i],end_list[i], max_results)
    json_response = connect_to_endpoint(search_url, headers, url, next_token)
    result_count = json_response['meta']['result_count']

    if 'next_token' in json_response['meta']:
        # Save the token to use for next request
        next_token = json_response['meta']['next_token']
        if result_count is not None and result_count > 0 and next_token is not None:
            append_to_csv(json_response)
            count += result_count
            print("Total Tweets collected: ", count)
            time.sleep(5)                
    # If no next token exists
    else:
        print("Data collection finished successfully")
        break

    # Delay between the requests
    time.sleep(5)

Total Tweets collected:  483
Total Tweets collected:  975


KeyboardInterrupt: ignored